# Delegation Unit Test
Performing verification of the delegation mechanism by comparing the event data to the results of cadCAD model.

In [1]:
import pandas as pd
import numpy as np
from decimal import Decimal
from decimal import *

# import os
# import sys
# module_path = os.path.abspath(os.path.join('..'))
# print(module_path)
# sys.path.append(module_path)
import model.parts.utils as utils
from model.sys_params import *
df = pd.read_pickle(r'experiment.p')


loaded GraphQL_data/SIstakeDelegateds.csv.
loaded GraphQL_data/SIstakeLockeds.csv.
loaded GraphQL_data/SIstakeWithdrawns.csv.
loaded GraphQL_data/SIallocationCloseds.csv.
loaded GraphQL_data/SIallocationCollecteds.csv.
loaded GraphQL_data/SIstakeDepositeds.csv.
loaded GraphQL_data/SIrewardsAssigneds.csv.


Configure environment

In [2]:
df.reset_index(inplace = True)
pd.set_option('display.max_rows', None)

In [3]:
delegation_events_dict = delegation_events[0]
print("EXPECTED TRUTH FROM DELEGATION EVENTS:")
for timestep, delegation_events in delegation_events_dict.items():
    for delegation_event in delegation_events:        
        print(f"{timestep}, {delegation_event['delegator']}, {delegation_event['shares']}")
    


EXPECTED TRUTH FROM DELEGATION EVENTS:
8, 0x63f8168f665f79101a0e0da2313cd1391e1c5e27, 1094.500000000000000000
8, 0x40f657e85bbb5cc7beb402da315c3fe1974442bf, 995.000000000000000000
12, 0x950d8f1e2e0ec438573a9540ca6f74913df0aeb1, 55222.500000000000000000
47, 0xcbb08e918e828bfd46a0e32e3365e80d7502c7d9, 2895.065106115432857526
51, 0xd45d0eb21cb9dbf86012cbd574fb66d1a2c6d800, 19300.434040769552383512
85, 0x8e0c59ba449ac228016dee5b1bb0990a8be7f8a6, 3753.469591583507658440


In [4]:
print("MODELED RESULTS")
delegation_event_shares = {}
for timestep, delegation_events in delegation_events_dict.items():
    for delegation_event in delegation_events:

        delegation_event_shares[timestep] = df.iloc[timestep-1].delegators[delegation_event['delegator']].shares
        print(f"{timestep}, {delegation_event['delegator']}, {delegation_event_shares[timestep]}")


MODELED RESULTS
8, 0x63f8168f665f79101a0e0da2313cd1391e1c5e27, 1094.500000000000000000
8, 0x40f657e85bbb5cc7beb402da315c3fe1974442bf, 995.0000000000000000000000000
12, 0x950d8f1e2e0ec438573a9540ca6f74913df0aeb1, 55222.50000000000000000000000
47, 0xcbb08e918e828bfd46a0e32e3365e80d7502c7d9, 2895.067010486121597421761460
51, 0xd45d0eb21cb9dbf86012cbd574fb66d1a2c6d800, 19349.82120910402185539210835
85, 0x8e0c59ba449ac228016dee5b1bb0990a8be7f8a6, 3764.318324385192642500632798


In [5]:
print("UNITTEST RESULTS")
for timestep, delegation_events in delegation_events_dict.items():
    for delegation_event in delegation_events:
        delegation_event_shares[timestep] = df.iloc[timestep-1].delegators[delegation_event['delegator']].shares
        print(f"Event: Timestep={timestep}, Delegator ID={delegation_event['delegator']}, Shares Granted={delegation_event['shares']}")
        print(f"Model: Timestep={timestep}, Delegator ID={delegation_event['delegator']}, Shares Granted={delegation_event_shares[timestep]}")
        print("Shares Equal?", delegation_event['shares'] == delegation_event_shares[timestep])
        print("Shares Within 1%?", 0.99 <= abs(delegation_event['shares'] / delegation_event_shares[timestep]) <= 1.01)
        print()


UNITTEST RESULTS
Event: Timestep=8, Delegator ID=0x63f8168f665f79101a0e0da2313cd1391e1c5e27, Shares Granted=1094.500000000000000000
Model: Timestep=8, Delegator ID=0x63f8168f665f79101a0e0da2313cd1391e1c5e27, Shares Granted=1094.500000000000000000
Shares Equal? True
Shares Within 1%? True

Event: Timestep=8, Delegator ID=0x40f657e85bbb5cc7beb402da315c3fe1974442bf, Shares Granted=995.000000000000000000
Model: Timestep=8, Delegator ID=0x40f657e85bbb5cc7beb402da315c3fe1974442bf, Shares Granted=995.0000000000000000000000000
Shares Equal? True
Shares Within 1%? True

Event: Timestep=12, Delegator ID=0x950d8f1e2e0ec438573a9540ca6f74913df0aeb1, Shares Granted=55222.500000000000000000
Model: Timestep=12, Delegator ID=0x950d8f1e2e0ec438573a9540ca6f74913df0aeb1, Shares Granted=55222.50000000000000000000000
Shares Equal? True
Shares Within 1%? True

Event: Timestep=47, Delegator ID=0xcbb08e918e828bfd46a0e32e3365e80d7502c7d9, Shares Granted=2895.065106115432857526
Model: Timestep=47, Delegator ID=0

# Conclusion
The number of shares granted by delegation in the first two events match the output of the cadCAD model exactly and then the next 3 are extremely close, diverging gradually.